# FACETpy Full Correction Workflow (Explicit Pipeline Syntax)

This tutorial builds the full correction workflow manually with `Pipeline([...])`, so you can see and control every processor step.

In [ ]:
from pathlib import Path

from facet import (
    AASCorrection,
    CutAcquisitionWindow,
    DownSample,
    EDFExporter,
    HighPassFilter,
    Loader,
    LowPassFilter,
    MedianArtifactCalculator,
    MetricsReport,
    Pipeline,
    RMSCalculator,
    RMSResidualCalculator,
    RawPlotter,
    SNRCalculator,
    TriggerAligner,
    TriggerDetector,
    UpSample,
    PasteAcquisitionWindow,
)

try:
    from facet import PCACorrection
except ImportError:
    PCACorrection = None

project_root = Path.cwd()
if not (project_root / "examples").exists() and (project_root.parent / "examples").exists():
    project_root = project_root.parent

input_file = project_root / "examples" / "datasets" / "NiazyFMRI.edf"
output_dir = project_root / "output" / "notebooks"
output_dir.mkdir(parents=True, exist_ok=True)

output_file = output_dir / "corrected_explicit_pipeline.edf"
plot_file = output_dir / "explicit_pipeline_before_after.png"

print(f"Input file: {input_file}")
assert input_file.exists(), f"Missing dataset: {input_file}"
print(f"PCACorrection available: {PCACorrection is not None}")

## Step 1: Define Core Correction Steps

These processors cover the core sequence: load, detect triggers, isolate acquisition, high-pass filter, upsample, align triggers, and perform AAS artifact subtraction.

In [ ]:
processors = [
    Loader(path=str(input_file), preload=True, artifact_to_trigger_offset=-0.005),
    TriggerDetector(regex=r"\\b1\\b"),
    CutAcquisitionWindow(),
    HighPassFilter(freq=1.0),
    UpSample(factor=10),
    TriggerAligner(ref_trigger_index=0, upsample_for_alignment=False),
    AASCorrection(window_size=30, correlation_threshold=0.975, realign_after_averaging=True),
]

print(f"Core steps configured: {len(processors)}")

## Step 2: Add Optional PCA Residual Cleanup

PCA correction can remove structured residual artifact after AAS. This cell adds it only when available in the current installation.

In [ ]:
if PCACorrection is not None:
    processors.append(PCACorrection(n_components=0.95, hp_freq=1.0))
    print("Added PCACorrection step.")
else:
    print("PCACorrection unavailable; continuing without PCA.")

## Step 3: Add Reconstruction, Evaluation, and Export

After correction, we return to original sampling rate, paste acquisition windows back, low-pass filter, compute quality metrics, create a comparison plot, and export EDF output.

In [ ]:
processors.extend([
    DownSample(factor=10),
    PasteAcquisitionWindow(),
    LowPassFilter(freq=70.0),
    SNRCalculator(),
    RMSCalculator(),
    RMSResidualCalculator(),
    MedianArtifactCalculator(),
    MetricsReport(),
    RawPlotter(
        mode="matplotlib",
        channel="Fp1",
        start=25.0,
        duration=20.0,
        overlay_original=True,
        show=False,
        auto_close=True,
        save_path=str(plot_file),
        title="Explicit Pipeline Workflow: Before vs After",
    ),
    EDFExporter(path=str(output_file), overwrite=True),
])

pipeline = Pipeline(processors, name="Explicit Full Correction Workflow")

## Step 4: Inspect Final Processor Order

Printing the step order helps verify exactly what will run before executing the full workflow.

In [ ]:
for idx, processor in enumerate(pipeline.processors, start=1):
    print(f"{idx:02d}. {processor.name}")

## Step 5: Execute the Full Pipeline

This runs all configured processors in sequence and returns a `PipelineResult` with runtime and quality metadata.

In [ ]:
result = pipeline.run()
result.print_summary()
result.print_metrics()

## Step 6: Inspect Context and Artifacts

The final context contains processed data, trigger metadata, and processing history for debugging and reporting.

In [ ]:
ctx = result.context
n_triggers = len(ctx.get_triggers()) if ctx.get_triggers() is not None else 0

print(f"Channels: {ctx.get_n_channels()}")
print(f"Triggers: {n_triggers}")
print(f"History steps: {len(ctx.get_history())}")
print(f"Output exists: {output_file.exists()}")
print(f"Plot exists: {plot_file.exists()}")